# Libraries Import

In [1]:
import pickle
import pandas as pd
import numpy as np
import json
import os

import operator
import math

import nltk
# from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

from gensim.models import Doc2Vec

from sklearn.externals import joblib

C:\Users\Victor\Anaconda3\envs\tensorflowCPU\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Functions used

In [2]:
#Read a json file
def open_file(fname):
    with open(fname) as jsonfile:
            resumes = json.load(jsonfile)
    return resumes

In [3]:
def create_arr_by_skills(resumes):
    job_desc=[]
    c=0
    #     print(puesto)
    skill=""
    for skills in resumes:
        skill=skill+" "+skills['skill_name']+"\n"#.replace(" ","_")
    skill=skill.lower()
    #     print(skill)
    #         cat=skills['skill_category']
    if len(skill)>0:
        job_desc.append(skill)
        c=c+1
    return job_desc  

In [4]:
#Sentence tokenization 
def token_separation(sDescription="",splitted=True,bSentence=False,bStopwords=False):
    
    sentenceTkn = []
    
    #Use of stopwords
    stoplist = []
    if bStopwords:
        stoplist = list(set(stopwords.words('english')))
        stoplist.extend(["–","/","&",".","$","(",")","!","?",";",":","'",",","“","”","’"])
    
    #Convert Description to sentence and Tokenization of words
    tnzr = nltk.tokenize
    if bSentence:
        sDescription=sDescription.replace("\\n","\n").replace(" \n ","\n").replace(" \n","\n").replace("\n ","\n").replace("\n",".\n<sen>").replace("..\n",".\n")
        sentences = sent_tokenize(sDescription)
        for sen in sentences:
            tnkstr= tnzr.word_tokenize(sen.replace("<sen>",""))
            
            i_offset = 0
            for i, t in enumerate(tnkstr):
                i -= i_offset
                if t == '#' and i > 0:
                    left = tnkstr[:i-1]
                    joined = [tnkstr[i - 1] + t]
                    right = tnkstr[i + 1:]
                    tnkstr = left + joined + right
                    i_offset += 1
    
            lSenTkn=[word for word in tnkstr if (word not in stoplist)]
            if len(lSenTkn)>0:
                if splitted:
                    sentenceTkn.append(lSenTkn)
                else:
                    sentenceTkn.extend(lSenTkn)
    else:
        sentences = sDescription
        tnkstr= tnzr.word_tokenize(sentences)   
        sentenceTkn = [word for word in tnkstr if (word not in stoplist)]

    return sentenceTkn

In [5]:
#Infering sentences
def infer_doc2vec(model_name,jobs_labeled):
    model = model_name

    lbl=[]
    emb=[]
    for job in jobs_labeled:
        model.random.seed(0)
        vecjob = model.infer_vector(doc_words=job)#, steps=20, alpha=0.025)
        emb.append(vecjob)
    
    docs_emb = {"Labels": lbl
               ,"Embedding": np.array(emb)}
    return emb

In [6]:
def dot_product(v1, v2):
    return sum(map(operator.mul, v1, v2))

def simil_cos(v1, v2):
    prod = dot_product(v1, v2)
    len1 = math.sqrt(dot_product(v1, v1))
    len2 = math.sqrt(dot_product(v2, v2))
    return prod / (len1 * len2)

In [7]:
def getClusterQualified(JobId,dPath,LinkedInEmb,model):
    Label = CatId2Job[JobId]
    print("Job Predicted: {}".format(Label))
    pathJob = os.path.join(dPath,"CompletedPreprocessing/"+Label.replace(" ","_"))
    
    fnameSkills = os.path.join(pathJob,"ClusterSkillDict_{}.p".format(Label.replace(" ","")))
    fnameSum = os.path.join(pathJob,"ClusterSkillSummaryDict_{}.p".format(Label.replace(" ","")))
    
    OverviewedSkills = pd.DataFrame.from_dict(pickle.load(open( fnameSkills, "rb" ) ))
    SumSkill = pd.DataFrame.from_dict(pickle.load(open( fnameSum, "rb" ) ))

    qClust=[]
    mainclusters=SumSkill['Cluster Label'].values
    for n,lsk in enumerate(LinkedInEmb):
        for mc in mainclusters:
            arrSkill=OverviewedSkills[OverviewedSkills['Cluster']==mc].values
            for dfs in arrSkill:
                emb=infer_doc2vec(model,token_separation(dfs[1],bSentence=True,bStopwords=True))
    #         print(emb)
                sim = simil_cos(np.array(lsk),emb[0])
                if sim > 0.95:
                    qClust.append(dfs[0])
                    break;
    lqClust= list(set(qClust))
    
    #To get the Qualified skills by the profile received
    QualSkills = SumSkill[SumSkill['Cluster Label'].isin(lqClust)][['Category','Skill']]
    NeededSkills = SumSkill[~SumSkill['Cluster Label'].isin(lqClust)][['Category','Skill']]
    print ("\n##################  Skills Qualified  #####################")
    print(QualSkills)
    print ("\n##################  Nedded Skills  #####################")
    print(NeededSkills)
    

# Processing

In [8]:
#Definition of the paths to be used
dPath= os.getcwd()
LinkedInpath= "C:\\Users\\Victor\\Documents\\LinkedIn Input"

In [13]:
#Definition of the files to be used
fNameTest = os.path.join(LinkedInpath,"TestCVLinkedin(31).json")

fDoc2VecModel = os.path.join(dPath,'Model/Doc2vecForSentences.model')
fNameModelSk = os.path.join(dPath,"Model/LogisticRegressionModelSkills.pkl")
fNameModelExp = os.path.join(dPath,"Model/LogisticRegressionModelExperience.pkl")
fNameCatj2Id = os.path.join(dPath,"Model/CatalogJobtoId.pkl")
fNameCatId2J = os.path.join(dPath,"Model/CatalogIdtoJob.pkl")

In [14]:
# load the items from disk
Doc2VecModel = Doc2Vec.load(fDoc2VecModel)
lrModelExp = joblib.load(fNameModelExp)
CatId2Job = pickle.load( open( fNameCatId2J, "rb" ) )

## Profile 1

In [21]:
testResumes = open_file(fNameTest)
Name=testResumes['Resume'][0]['name']
actualJob= testResumes['Resume'][0]['job']

Experience = token_separation(testResumes['Resume'][0]['summary'].lower(),splitted=False,bSentence=True,bStopwords=True)
expEmbedding=infer_doc2vec(Doc2VecModel,[Experience])

y = lrModelExp.predict(expEmbedding)#1,9TA,14,16,21,8

Skills = create_arr_by_skills(testResumes['Resume'][0]['skills_endorsment'])
SkillsTkn =  token_separation(Skills[0],bSentence=True,bStopwords=True)
SkillEmb= infer_doc2vec(Doc2VecModel,SkillsTkn)

print ("Name: {}".format(Name))
print ("Actual Job: {}".format(actualJob))
lqClust = getClusterQualified(y[0],dPath,SkillEmb,Doc2VecModel)

print ("\n\nTo levelup in this area...\n\n")

lqClust = getClusterQualified(y[0]+1,dPath,SkillEmb,Doc2VecModel)

Name: Bruce Jachens
Actual Job: Sr. Project Manager
Job Predicted: senior project manager or equivalent

##################  Skills Qualified  #####################
   Category                  Skill
0      must     project management
1       avg      business analysis
2       avg  requirements analysis
3       avg               strategy
5       avg       business process
7       avg    agile methodologies
8       avg     program management
10      avg      vendor management
13     plus                   sdlc
15     plus            integration
17     plus  business intelligence

##################  Nedded Skills  #####################
   Category                  Skill
4       avg       project planning
6       avg      change management
9       avg        team leadership
11      avg             leadership
12     plus        risk management
14     plus             consulting
16     plus        team management
18     plus       project delivery
19     plus   business development
20     

## Profile 2

In [25]:
testResumes = open_file(fNameTest)
Name=testResumes['Resume'][1]['name']
actualJob= testResumes['Resume'][1]['job']

Experience = token_separation(testResumes['Resume'][1]['summary'].lower(),splitted=False,bSentence=True,bStopwords=True)
expEmbedding=infer_doc2vec(Doc2VecModel,[Experience])

y = lrModelExp.predict(expEmbedding)#1,9TA,14,16,21,8

Skills = create_arr_by_skills(testResumes['Resume'][1]['skills_endorsment'])
SkillsTkn =  token_separation(Skills[0],bSentence=True,bStopwords=True)
SkillEmb= infer_doc2vec(Doc2VecModel,SkillsTkn)

print ("Name: {}".format(Name))
print ("Actual Job: {}".format(actualJob))
print ("summary: {}".format(testResumes['Resume'][1]['summary']))
lqClust = getClusterQualified(y[0],dPath,SkillEmb,Doc2VecModel)

print ("\n\nTo levelup in this area...\n\n")

lqClust = getClusterQualified(y[0]+1,dPath,SkillEmb,Doc2VecModel)

Name: César Martínez
Actual Job: Developer Sr
summary: Cesar is a Sr.Net professional with 10 years of experience , He has developing web applications and desktop application using Microsoft technologies He has working in the full Software Development Life Cycle including Requirements Analysis, High Level Design, Detailed Design, Build, Testing, Security, Acceptance and Installation, Maintenance and Production Support carried out several .Net roles as a technical leader, lead developer and technical consultant in the TI, Government, financial, Entertainment and media sectors. As a .NET Technical Lead/Architect he also has experience on each activity of the SDLC such as requirements gathering and analysis with uses cases and user stories, OO analysis and design using severalUML diagrams, OO developing with design patterns (such as Singleton, Abstract Factory and Factory Method, Dependency Injection,Memento, MVC,MVVM), functional testing with Mercury Quality Center 9.0, Test Manager, QT 

## Profile 3

In [35]:
testResumes = open_file(fNameTest)
Name=testResumes['Resume'][2]['name']
actualJob= testResumes['Resume'][2]['job']

Experience = token_separation(testResumes['Resume'][2]['summary'].lower(),splitted=False,bSentence=True,bStopwords=True)
expEmbedding=infer_doc2vec(Doc2VecModel,[Experience])

y = lrModelExp.predict(expEmbedding)#1,9TA,14,16,21,8

Skills = create_arr_by_skills(testResumes['Resume'][2]['skills_endorsment'])
SkillsTkn =  token_separation(Skills[0],bSentence=True,bStopwords=True)
SkillEmb= infer_doc2vec(Doc2VecModel,SkillsTkn)

print ("Name: {}".format(Name))
print ("Actual Job: {}".format(actualJob))
print ("summary: {}".format(testResumes['Resume'][2]['summary']))
lqClust = getClusterQualified(y[0],dPath,SkillEmb,Doc2VecModel)

print ("\n\nTo levelup in this area...\n\n")

lqClust = getClusterQualified(y[0]+1,dPath,SkillEmb,Doc2VecModel)

Name: Guillermo Romero
Actual Job: Android Engineer
summary: Develop Software able to process high volumes of data and huge demand. Propose and build innovative ways in which the new technologies could help companies to increase their profit.
Job Predicted: senior consultant or equivalent

##################  Skills Qualified  #####################
   Category            Skill
0      must              sql
1      must       javascript
2       avg             java
4       avg             html
5       avg              css
6       avg              xml
9       avg            mysql
11      avg  web development
14     plus           jquery
21     plus              jsp

##################  Nedded Skills  #####################
   Category                 Skill
3       avg  software development
7       avg    project management
8       avg       team leadership
10      avg          web services
12     plus                 scrum
13     plus      web applications
15     plus      microsoft office


## Profile 4

In [39]:
testResumes = open_file(fNameTest)
Name=testResumes['Resume'][3]['name']
actualJob= testResumes['Resume'][3]['job']

Experience = token_separation(testResumes['Resume'][3]['summary'].lower(),splitted=False,bSentence=True,bStopwords=True)
expEmbedding=infer_doc2vec(Doc2VecModel,[Experience])

y = lrModelExp.predict(expEmbedding)#1,9TA,14,16,21,8

Skills = create_arr_by_skills(testResumes['Resume'][3]['skills_endorsment'])
SkillsTkn =  token_separation(Skills[0],bSentence=True,bStopwords=True)
SkillEmb= infer_doc2vec(Doc2VecModel,SkillsTkn)

print ("Name: {}".format(Name))
print ("Actual Job: {}".format(actualJob))
print ("summary: {}".format(testResumes['Resume'][3]['summary'].replace ("\\n","\n")))
lqClust = getClusterQualified(y[0],dPath,SkillEmb,Doc2VecModel)

print ("\n\nTo levelup in this area...\n\n")

lqClust = getClusterQualified(y[0]+1,dPath,SkillEmb,Doc2VecModel)

Name: Meleneal Cameron
Actual Job: Web Developer/Lead Campaign Specialist
summary: I am a driven professional with over ten years of experience and I am proficient in data management, web development, digital marketing, and content management. I am known for my excellent interpersonal skills and my ability to multitask and meet deadlines. 
WEB DEVELOPMENT 
HTML, CSS, JavaScript, PHP
CONTENT MANAGEMENT 
Wordpress, Joomla, Convio, Omni Update
ESPs
Convio, SalsaLabs, Responsys, Blackbaud
ANALYTICS 
SEO, Web Analytics, Target Analytics, Research Point, NOZA
DATABASE ADMINISTRATION
SQL, Server Enterprise Manager, Blackbaud Management Console, Raiser’s Edge, Convio, iMIS, IBM AS 400, Paradox, Microsoft Access 
SOFTWARE APPLICATIONS 
Adobe Dreamweaver, Photoshop, SEO Powersuite, Crystal Reports
Job Predicted: it consultant or equivalent

##################  Skills Qualified  #####################
  Category               Skill
2      avg  project management
5     plus                html

###